### Loading libraries

In [ ]:
library(tidyverse)
library(tidymodels)
library(gridExtra)

### Utility functions

In [ ]:
fig <- function(width, heigth){
     options(repr.plot.width = width, repr.plot.height = heigth)
}

### Loading data

In [ ]:
path <- ""

names <- read.table(paste(path, "spotify-names.txt", sep = ""),header = TRUE)
songs <- read.table(paste(path, "spotify-extr.txt", sep = ""),
                    sep = " ", header = TRUE) %>%
    as_tibble() %>%
    mutate(name = names$x,
           key = factor(key),
           mode = factor(mode, levels=c(0,1), labels=c('minor', 'major')),
           pop.class = factor(pop.class)) %>%
    relocate(c(pop.class, popularity)) %>%
    relocate(c(key, mode, name), .after=last_col())

songs.quant <- songs %>% select(popularity:tempo)

head(songs)

## Exploratory statistics

In [ ]:
fig(12,8)
songs.quant %>%
    pivot_longer(cols=everything(), names_to='variable', values_to='value') %>%
ggplot() +
    geom_histogram(aes(value), fill='#2FD565', color='#000000', bins=30) +
    facet_wrap(~variable, scales='free')

In [ ]:
fig(12,6)
songs %>%
    filter(popularity > 0) %>%
ggplot() +
    geom_histogram(aes(popularity), fill='#2FD565', color='#000000', binwidth = 5) +
    scale_x_continuous(breaks=seq(0,100, by=10))

In [ ]:
fig(8,6)
songs %>%
    mutate(log.duration=log(duration)) %>%
ggplot() +
    geom_histogram(aes(log.duration), fill='#2FD565', color='#000000', bins=50)

In [ ]:
LOG_DURATION <- FALSE
if (!LOG_DURATION) {
    songs <- songs %>%
        mutate(duration=log(duration))
    LOG_DURATION <- TRUE
}

In [ ]:
fig(15,10)
songs.quant %>%
    filter(popularity>0) %>%
    mutate(duration=log(duration)) %>%
    pivot_longer(cols=!popularity, names_to='variable', values_to='value') %>%
ggplot() +
    geom_point(aes(value, popularity), size=.7, alpha=.3) + 
    facet_wrap(~variable, scales='free')

In [ ]:
fig(10,8)
songs %>%
ggplot(aes(year %>% factor(), popularity)) +
    geom_boxplot(fill='#2FD565', varwidth = TRUE) +
    scale_x_discrete(breaks=seq(1920, 2020, by=10), name='Year') +
    labs(y='Popularity') +
    theme(text=element_text(size=14))

In [ ]:
fig(15,10)
songs %>%
    filter(popularity>0) %>%
    mutate(duration=log(duration)) %>%
    select(c(pop.class, valence:tempo)) %>%
    pivot_longer(cols=!pop.class, names_to='variable', values_to='value') %>%
ggplot() +
    #geom_jitter(aes(value, pop.class), size=.7, alpha=.3) + 
    geom_boxplot(aes(value, pop.class)) +
    facet_wrap(~variable, scales='free') +
    scale_y_discrete(limits=rev)

In [ ]:
library(corrplot)
cormat <- cor(songs.quant)
corrplot(cormat, method="ellipse", col=colorRampPalette(c("blue", "white", "red"))(200))

In [ ]:
fig(8,6)
songs %>%
ggplot() +
    geom_boxplot(aes(x=reorder(key, -popularity, FUN=median), y=popularity), fill='#2FD565', color='#000000') +
    labs(x='Key', y='Popularity')


songs %>%
ggplot() +
    geom_bar(stat='count', aes(reorder(key, -popularity, FUN=median)), fill='#2FD565', color='#000000') +
    labs(x='Key', y='Count')

In [ ]:
fig(8,6)
songs %>%
ggplot() +
    geom_boxplot(aes(mode, popularity), fill='#2FD565', color='#000000') +
    labs(x='Mode', y='Popularity')

In [ ]:
library(ggmosaic)
songs %>%
ggplot() +
    geom_mosaic(aes(product(mode, key)), fill='#2FD565')

## Principal component analysis

In [ ]:
library(FactoMineR)
library(factoextra)
res.pca <- songs.quant %>% 
    select(!popularity) %>% 
    PCA(ncp=11)

In [ ]:
fig(12,5)


pc_eig <- tibble(pc=c(1:11), eig=res.pca$eig[1:11,2], cumeig=res.pca$eig[1:11,3])
g1 <- pc_eig %>%
ggplot(aes(pc, eig)) + 
    geom_bar(stat="identity", fill='#2FD565') +
    geom_line() + 
    geom_point() +
    scale_x_continuous(breaks=c(1:11), labels=paste('PC', c(1:11), sep=''), minor_breaks=NULL) +
    labs(x='', y='Percentage of variance', title='Percentage of variance by principal component') +
    annotate('text', x=c(1:11)+.4, y=pc_eig$eig+1.1, label=paste(round(pc_eig$eig,1), '%', sep='')) +
    theme(text=element_text(size=14))

g2 <- pc_eig %>%
ggplot(aes(pc, cumeig)) + 
    geom_bar(stat="identity", fill='#2FD565') +
    geom_line() + 
    geom_point() +
    scale_x_continuous(breaks=c(1:11), labels=paste('PC', c(1:11), sep=''), minor_breaks=NULL) +
    labs(x='', y='Percentage of variance', title='Cumulative percentage of variance') +
    #annotate('text', x=c(1:11)-.1, y=pc_eig$cumeig+6, label=paste(round(pc_eig$cumeig,1), '%', sep='')) +
    theme(text=element_text(size=14))

grid.arrange(g1, g2, nrow=1)

In [ ]:
pc_eig

In [ ]:
fig(8,6)
fviz_pca_var(res.pca, col.var="contrib") +
    scale_color_gradient(low="black", high="green") +
    labs(x='PC1 (31.3%)', y='PC2 (14.9%)', color='Contribution') +
    guides(color='none') +
    theme_gray() + theme(text=element_text(size=14))

In [ ]:
name_func <- function(name) {
    paste('PC', substring(name, first=5), sep='')
}

pca.ind <- as_tibble(res.pca$ind$coord) %>%
    rename_with(name_func, everything()) %>%
    mutate(pop.class=songs$pop.class)

gg_color_hue <- function(n) {
  hues = seq(15, 375, length = n + 1)
  hcl(h = hues, l = 65, c = 100)[1:n]
}
colors <- gg_color_hue(4)

fig(10,8)
ggplot(pca.ind) + 
    geom_point(aes(x=PC1, y=PC2, color=pop.class), alpha=.5, shape=19) +
    scale_x_continuous(limits=c(-6,4)) +
    scale_y_continuous(limits=c(-5,5)) +
    guides(color = guide_legend(override.aes = list(size = 10))) +
    labs(x='PC1 (31.3%)', y='PC2 (14.9%)', color="Popularity class") +
    theme(text=element_text(size=14))

# Classification models

In [ ]:
library(rsample)

train_ratio <- .8
data_split <- songs %>%
    initial_split(strata = pop.class, prop = train_ratio)

songs_train <- training(data_split) %>%
    select(!c(popularity, name))

songs_test  <- testing(data_split) %>%
    select(!c(popularity, name))

In [ ]:
songs_cv <- rsample::vfold_cv(songs_train, v=10)

In [ ]:
pop_count <- songs_test %>%
    group_by(pop.class) %>%
    count() %>%
    ungroup() %>%
    .[c(rep(1, 4), rep(2, 4), rep(3, 4), rep(4, 4)),]

plot_conf_mat <- function(model_results) {
    
    conf_mat <- model_results %>%
        group_by(pop.class, .pred_class) %>%
        count(.pred_class, pop.class, .drop=FALSE) %>%
        ungroup() %>%
        mutate(pop_count=pop_count$n) %>%
        mutate(freq=n / pop_count) %>%
        mutate(correct=ifelse(.pred_class==pop.class, 1, -1))
    
    cell_labs <- paste(round(100*conf_mat$freq), '%\n(N=', conf_mat$n, ')', sep='')
    cell_labs <- paste(conf_mat$n, '\n(',round(100*conf_mat$freq), '%)', sep='')
    
    conf_mat %>%
    ggplot(aes(.pred_class, pop.class)) +
    geom_tile(aes(fill=freq*correct), color='black', size=.3) +
    geom_text(aes(label=cell_labs), color='black', size=8) +
    scale_fill_gradient2(low='pink', mid='white', high='#2FD565') +
    scale_y_discrete(limits=rev, name='Popularity class') +
    scale_x_discrete(position='top', name='Predicted class') +
    coord_equal() +
    theme_minimal() +
    theme(legend.position = "none",
          text=element_text(size=26),
          axis.text=element_text(size=30))
}

## Logistic regression

In [ ]:
library(glmnet)

### Without regularization

In [ ]:
log_reg <- multinom_reg(penalty = 0) %>%
    set_engine('glmnet')

In [ ]:
log_reg_fit <-
    log_reg %>%
    fit(pop.class ~ ., 
        data=songs_train)


tidy(log_reg_fit) %>% 
    filter(term != '(Intercept)') %>%
ggplot(aes(term, class)) +
    geom_tile(aes(fill=estimate)) +
    scale_fill_gradient2(low = "blue",
                         mid = "white",
                         high = "red",
                         midpoint = 0) +
    coord_equal() +
    theme(text=element_text(size=16), axis.text.x = element_text(angle = 90))

In [ ]:
log_reg_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(log_reg_fit, new_data = songs_test %>% select(-pop.class))
  )

In [ ]:
plot_conf_mat(log_reg_results)

log_reg_results %>%
    yardstick::accuracy(pop.class, .pred_class)

### With regularization

In [ ]:
#log_reg_grid <- tibble(penalty = 10^seq(-3, 0, length.out = 30))
#log_reg_grid <- tibble(penalty = 10^seq(-3, -.3, length.out = 30))
log_reg_grid <- expand.grid(penalty = 10^seq(-3, 0, length.out = 20), mixture=seq(0,1, length = 5))

log_reg_rec <- recipe(pop.class ~ ., data = songs_train) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>%
    step_dummy(all_nominal(), -all_outcomes())
    step_

log_reg_mod <- multinom_reg(penalty = tune(), mixture = tune()) %>%
    set_engine("glmnet")

log_reg_wf <- workflow() %>%
    add_recipe(log_reg_rec) %>%
    add_model(log_reg_mod)

In [ ]:
log_reg_fit <- log_reg_wf %>%
    tune_grid(
        resamples = songs_cv,
        grid = log_reg_grid,
        metrics = metric_set(accuracy, roc_auc),
        control = control_grid(verbose = TRUE)
    )

In [ ]:
fig(10,8)
log_reg_fit %>%
    collect_metrics() %>%
    ggplot(aes(penalty, mean, color = factor(mixture))) +
    #geom_errorbar(aes(ymin = mean - std_err,
    #                  ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Penalty', y='Metric', color='Mixture') +
    theme(text=element_text(size=16))

In [ ]:
log_reg_best <- log_reg_fit %>%
    select_by_one_std_err(metric="accuracy", desc(penalty))
    #select_best(metric="accuracy")
    #select_by_one_std_err(metric="rmse", penalty)

log_reg_wf %>% 
    finalize_workflow(log_reg_best) %>%
    fit(songs_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    filter(term != '(Intercept)') %>%
    #select(term, estimate) %>%
    #arrange(desc(abs(estimate))) %>%
ggplot(aes(term, class)) +
    geom_tile(aes(fill=estimate)) +
    scale_fill_gradient2(low = "blue",
                           mid = "white",
                           high = "red",
                           midpoint = 0) +
    coord_equal() +
    theme(text=element_text(size=16), axis.text.x = element_text(angle = 90))

In [ ]:
log_reg_wf %>% 
    finalize_workflow(log_reg_best) %>%
    fit(songs_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    filter(abs(estimate) > 0) %>%
    filter(term != '(Intercept)')

In [ ]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")
lin_reg_wf %>% 
    finalize_workflow(lin_reg_best) %>%
    fit(songs_reg_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    filter(abs(estimate) > 0) %>%
    select(term, estimate) %>%
    xtable(type = "latex") %>%
print(file = "lm.tex")

In [ ]:
log_reg_best_fit <- log_reg_wf %>%
    finalize_workflow(log_reg_best) %>%
    fit(data=songs_train)

log_reg_results <- log_reg_best_fit %>%
    predict(new_data = songs_test) %>%
    bind_cols(songs_test, .) %>%
    select(pop.class, .pred_class)

In [ ]:
plot_conf_mat(log_reg_results)

log_reg_results %>%
    yardstick::accuracy(pop.class, .pred_class)

## K-nearest neighbours

In [ ]:
knn_grid <- tibble(neighbors = seq(10, 70, by=5))

knn_rec <- recipe(pop.class ~ ., data = songs_train) %>%
    step_dummy(all_nominal(), -all_outcomes())

knn_mod <- nearest_neighbor(neighbors=tune()) %>%
    set_mode("classification") %>%
    set_engine("kknn")

knn_wf <- workflow() %>%
    add_recipe(knn_rec) %>%
    add_model(knn_mod)

In [ ]:
knn_fit <- knn_wf %>%
    tune_grid(
        resamples = songs_cv,
        grid = knn_grid
    )

In [ ]:
knn_fit %>%
    collect_metrics()

In [ ]:
fig(10,8)
knn_fit %>%
    collect_metrics() %>%
    ggplot(aes(neighbors, mean, color = .metric)) +
    geom_errorbar(aes(ymin = mean - std_err,
                      ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Cost', y='Metric') +
    theme(legend.position = "none",
          text=element_text(size=16))

In [ ]:
knn_best <- knn_fit %>%
    #select_by_one_std_err(metric="accuracy", desc(cost_complexity))
    select_best(metric="accuracy")
    #select_by_one_std_err(metric="rmse", penalty)

In [ ]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")
lin_reg_wf %>% 
    finalize_workflow(lin_reg_best) %>%
    fit(songs_reg_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    filter(abs(estimate) > 0) %>%
    select(term, estimate) %>%
    xtable(type = "latex") %>%
print(file = "lm.tex")

In [ ]:
knn_best_fit <- knn_wf %>% 
    finalize_workflow(knn_best) %>%
    fit(data=songs_train)

knn_results <- knn_best_fit %>%
    predict(new_data = songs_test) %>%
    bind_cols(songs_test, .) %>%
    select(pop.class, .pred_class)

In [ ]:
plot_conf_mat(knn_results)

knn_results %>%
    yardstick::accuracy(pop.class, .pred_class)

## SMV

### Linear kernel

In [ ]:
svm_lin_grid <- tibble(cost = 10^seq(-3,1, length.out = 10))

svm_lin_rec <- recipe(pop.class ~ ., data = songs_train) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>%
    step_select(-all_nominal_predictors())
    #step_dummy(all_nominal(), -all_outcomes())

svm_lin_mod <- svm_poly(degree=1, cost=tune()) %>%
    set_mode("classification") %>%
    set_engine("kernlab")

svm_lin_wf <- workflow() %>%
    add_recipe(svm_lin_rec) %>%
    add_model(svm_lin_mod)

In [ ]:
svm_lin_fit <- svm_lin_wf %>%
    tune_grid(
        resamples = songs_cv,
        grid = svm_lin_grid,
        control = control_grid(verbose = TRUE)
    )

In [ ]:
svm_lin_fit %>%
    collect_metrics()

In [ ]:
fig(10,8)
svm_lin_fit %>%
    collect_metrics() %>%
    ggplot(aes(cost, mean, color = .metric)) +
    geom_errorbar(aes(ymin = mean - std_err,
                      ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Cost', y='Metric') +
    theme(legend.position = "none",
          text=element_text(size=16))

In [ ]:
svm_lin_best <- svm_lin_fit %>%
    #select_by_one_std_err(metric="accuracy", desc(cost_complexity))
    select_best(metric="roc_auc")
    #select_by_one_std_err(metric="rmse", penalty)

In [ ]:
svm_lin_best_fit <- svm_lin_wf %>% 
    finalize_workflow(svm_lin_best) %>%
    fit(data=songs_train)

#svm_lin_results <- svm_lin_best_fit %>%
#    predict(new_data = songs_test) %>%
#    bind_cols(songs_test, .) #%>%
    #select(pop.class, .pred_class)

In [ ]:
svm_lin_best_fit %>% 
    predict(new_data = songs_test)

In [ ]:
svm_lin_results %>%
    yardstick::accuracy(.pred_class, pop.class)

In [ ]:
plot_conf_mat(svm_lin_results)

In [ ]:
songs_rec <-
    recipe(pop.class ~ ., data = songs_train) %>%
    step_normalize(all_numeric(), -all_outcomes())

In [ ]:
formula_res <-
    svm_mod %>% 
    tune_grid(
        songs_rec,
        resamples = songs_cv,
        grid = tibble(cost=seq(.1, 2, length=2))
    )
#formula_res

In [ ]:
estimates <- collect_metrics(formula_res)
estimates %>%
    #filter(.metric=='roc_auc') %>%
    select(cost, mean, std_err, .metric) %>%
ggplot(aes(cost, mean)) + 
    #geom_smooth(aes(ymax=mean+2*std_err, ymin=mean-2*std_err), stat='identity', color='#2FD565') + 
    geom_line(color='#2FD565') +
    geom_errorbar(aes(ymax=mean+2*std_err, ymin=mean-2*std_err), color='#2FD565') +
    scale_y_continuous(limits=c(0.5,1)) +
    facet_wrap(~.metric)

In [ ]:
formula_res %>%
    show_best() %>%
    slice(1) %>%
    select(cost, rbf_sigma)

In [ ]:
svm_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(s_fit, new_data = songs_test %>% select(-pop.class))
  )

In [ ]:
svm_tidy <- svm_rbf(cost=params$cost, rbf_sigma=params$rbf_sigma) %>%
    set_mode('classification') %>%
    set_engine('kernlab')

In [ ]:
svm_tidy_fit <-
    svm_tidy %>%
    fit(pop.class ~ ., 
        data=songs_train)

#tidy(svm_tidy_fit) %>% filter(estimate > 0)

In [ ]:
svm_tidy_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(svm_tidy_fit, new_data = songs_test %>% select(-pop.class))
  )

In [ ]:
svm_tidy_results %>%
    conf_mat(truth=pop.class, estimate=.pred_class)

svm_tidy_results %>%
    yardstick::precision(pop.class, .pred_class)

### Polynomial kernel

In [ ]:
svm_poly_grid <- expand.grid(cost = 10^seq(-3,1, length.out = 5), degree = c(1:3))

svm_poly_rec <- recipe(pop.class ~ ., data = songs_train) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>%
    step_dummy(all_nominal(), -all_outcomes())

svm_poly_mod <- svm_poly(degree= tune(), cost=tune()) %>%
    set_mode("classification") %>%
    set_engine("kernlab")

svm_poly_wf <- workflow() %>%
    add_recipe(svm_poly_rec) %>%
    add_model(svm_poly_mod)

In [ ]:
svm_poly_fit <- svm_poly_wf %>%
    tune_grid(
        resamples = songs_cv,
        grid = svm_poly_grid
    )

In [ ]:
svm_poly_fit %>%
    collect_metrics()

In [ ]:
fig(10,8)
svm_lin_fit %>%
    collect_metrics() %>%
    ggplot(aes(cost, mean, color = .metric)) +
    geom_errorbar(aes(ymin = mean - std_err,
                      ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Cost', y='Metric') +
    theme(legend.position = "none",
          text=element_text(size=16))

In [ ]:
svm_lin_best <- svm_lin_fit %>%
    #select_by_one_std_err(metric="accuracy", desc(cost_complexity))
    select_best(metric="roc_auc")
    #select_by_one_std_err(metric="rmse", penalty)

In [ ]:
svm_lin_best_fit <- svm_lin_wf %>% 
    finalize_workflow(svm_lin_best) %>%
    fit(data=songs_train)

svm_lin_results <- svm_lin_best_fit %>%
    predict(new_data = songs_test) %>%
    bind_cols(songs_test, .) %>%
    select(pop.class, .pred_class)

In [ ]:
svm_lin_results %>%
    yardstick::accuracy(.pred_class, pop.class)

In [ ]:
plot_conf_mat(svm_lin_results)

### Radial kernel

In [ ]:
svm_rad_grid <- tibble(cost = 10^seq(-4,1, length.out = 5))

svm_lin_rec <- recipe(pop.class ~ ., data = songs_train) %>%
    step_dummy(all_nominal(), -all_outcomes())

svm_lin_mod <- svm_poly(degree=1, cost=tune()) %>%
    set_mode("classification") %>%
    set_engine("kernlab")

svm_lin_wf <- workflow() %>%
    add_recipe(svm_lin_rec) %>%
    add_model(svm_lin_mod)

In [ ]:
svm_lin_fit <- svm_lin_wf %>%
    tune_grid(
        resamples = songs_cv,
        grid = svm_lin_grid
    )

In [ ]:
svm_lin_fit %>%
    collect_metrics()

In [ ]:
fig(10,8)
svm_lin_fit %>%
    collect_metrics() %>%
    ggplot(aes(cost, mean, color = .metric)) +
    geom_errorbar(aes(ymin = mean - std_err,
                      ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Cost', y='Metric') +
    theme(legend.position = "none",
          text=element_text(size=16))

In [ ]:
tree_best <- tree_fit %>%
    select_by_one_std_err(metric="accuracy", desc(cost_complexity))
    #select_best(metric="accuracy")
    #select_by_one_std_err(metric="rmse", penalty)

In [ ]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")
lin_reg_wf %>% 
    finalize_workflow(lin_reg_best) %>%
    fit(songs_reg_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    filter(abs(estimate) > 0) %>%
    select(term, estimate) %>%
    xtable(type = "latex") %>%
print(file = "lm.tex")

In [ ]:
tree_best_fit <- tree_wf %>% 
    finalize_workflow(tree_best) %>%
    fit(data=songs_train)

tree_results <- tree_best_fit %>%
    predict(new_data = songs_test) %>%
    bind_cols(songs_test, .) %>%
    select(pop.class, .pred_class)

In [ ]:
tree_results %>%
    mutate(correct=pop.class == .pred_class) %>%
    summarise(accuracy=sum(correct==TRUE) / n)

In [ ]:
plot_conf_mat(tree_results)

In [ ]:
rad_svm <- train(pop.class ~., 
                 data = songs_train, 
                 method = "svmRadial", 
                 trControl = train_control,  
                 preProcess = c("center","scale"),
                 tuneGrid = expand.grid(C = seq(.5, 2, length = 3), 
                                        sigma = seq(0.001, 0.2, length = 6)))

rad_svm

In [ ]:
plot(rad_svm)

In [ ]:
str(rad_svm$bestTune)

In [ ]:
rad_svm_mod <- svm_rbf(mode='classification', cost=rad_svm$bestTune$C, rbf_sigma=rad_svm$bestTune$sigma) %>%
    set_engine("kernlab")
rad_svm_mod

In [ ]:
rad_svm_fit <-
    rad_svm_mod %>%
    fit(pop.class ~ ., 
        data=songs_train)

tidy(rad_svm_fit)

In [ ]:
rad_svm_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(rad_svm_fit, new_data = songs_test %>% select(-pop.class))
  )
head(rad_svm_results)

In [ ]:
rad_svm_results %>%
    conf_mat(truth=pop.class, estimate=.pred_class)

rad_svm_results %>%
    yardstick::precision(truth=pop.class, estimate=.pred_class)

## Decision tree

In [ ]:
tree_grid <- tibble(cost_complexity = 10^seq(-3, 0, length.out = 30))

tree_rec <- recipe(pop.class ~ ., data = songs_train) #%>%
    #step_normalize(all_numeric(), -all_outcomes()) %>%
    #step_dummy(all_nominal(), -all_outcomes())

tree_mod <- decision_tree(cost_complexity = tune(), tree_depth=5) %>%
    set_mode("classification") %>%
    set_engine("rpart")

tree_wf <- workflow() %>%
    add_recipe(tree_rec) %>%
    add_model(tree_mod)

In [ ]:
tree_fit <- tree_wf %>%
    tune_grid(
        resamples = songs_cv,
        grid = tree_grid,
        control = control_grid(verbose=TRUE)
    )

In [ ]:
tree_fit %>%
    collect_metrics()

In [ ]:
fig(10,8)
tree_fit %>%
    collect_metrics() %>%
    ggplot(aes(cost_complexity, mean, color = .metric)) +
    geom_errorbar(aes(ymin = mean - std_err,
                      ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Cp penalty', y='Metric') +
    theme(legend.position = "none",
          text=element_text(size=16))

In [ ]:
tree_best <- tree_fit %>%
    #collect_metrics() %>%
    #arrange(.metric, desc(mean), cost_complexity) %>%
    #slice(16)
    select_by_one_std_err(metric="accuracy", desc(cost_complexity))
    #select_best(metric="accuracy")
    #select_by_one_std_err(metric="rmse", penalty)
#tree_best
#tree_fit %>%
#    collect_metrics() %>%
#    arrange(.metric, desc(mean), cost_complexity) %>% mutate(row=row_number())

In [ ]:
tree_best

In [ ]:
tree_best_fit <- tree_wf %>% 
    finalize_workflow(tree_best) %>%
    fit(data=songs_train)

tree_results <- tree_best_fit %>%
    predict(new_data = songs_test) %>%
    bind_cols(songs_test, .) %>%
    select(pop.class, .pred_class)

In [ ]:
tree_results %>%
    accuracy(pop.class, .pred_class)

In [ ]:
plot_conf_mat(tree_results)

In [ ]:
library(rpart.plot)
rpart.plot(tree_best_fit$fit$fit$fit)

In [ ]:
library(keras)

In [ ]:
nn_grid <- tibble(epochs=c(30, 50, 100))

nn_rec <- recipe(pop.class ~ ., data = songs_train) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>%
    step_dummy(all_nominal(), -all_outcomes())

nn_mod <- mlp(hidden_units = 8, epochs = tune()) %>%
    set_mode("classification") %>%
    set_engine("keras")

nn_wf <- workflow() %>%
    add_recipe(nn_rec) %>%
    add_model(nn_mod)

In [ ]:
nn_fit <- nn_wf %>%
    tune_grid(
        resamples = songs_cv,
        grid = nn_grid,
        control = control_grid(verbose=TRUE)
    )

In [ ]:
nn_fit %>%
    collect_metrics()

In [ ]:
fig(10,8)
nn_fit %>%
    collect_metrics() %>%
    ggplot(aes(epochs, mean, color = .metric)) +
    geom_errorbar(aes(ymin = mean - std_err,
                      ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Cp penalty', y='Metric') +
    theme(legend.position = "none",
          text=element_text(size=16))

In [ ]:
nn_best <- nn_fit %>%
    select_best(metric="accuracy")
    #select_by_one_std_err(metric="rmse", penalty)

In [ ]:
nn_best_fit <- nn_wf %>% 
    finalize_workflow(nn_best) %>%
    fit(data=songs_train)

nn_results <- nn_best_fit %>%
    predict(new_data = songs_test) %>%
    bind_cols(songs_test, .) %>%
    select(pop.class, .pred_class)

In [ ]:
nn_results %>%
    yardstick::accuracy(.pred_class, pop.class)

In [ ]:
plot_conf_mat(nn_results)

In [ ]:
nn_mod <- mlp(hidden_units = 16, epochs=300) %>%
    set_mode("classification") %>%
    set_engine("keras", verbose=2)

nn_wf <- workflow() %>%
    add_recipe(nn_rec) %>%
    add_model(nn_mod)

In [ ]:
nn_mod_fit <- nn_wf %>%
    fit(data=songs_train)

In [ ]:
nn_mod_results <- nn_mod_fit %>%
    predict(new_data = songs_test) %>%
    bind_cols(songs_test, .) %>%
    select(pop.class, .pred_class)

In [ ]:
nn_mod_results %>%
    yardstick::accuracy(pop.class, .pred_class)

fig(6,6)
plot_conf_mat(nn_mod_results)

# Regression models

#### With year

In [ ]:
train_ratio <- .8
reg_data_split <- songs %>%
    initial_split(prop = train_ratio)

songs_reg_train <- training(reg_data_split) %>% 
    #select(!c(pop.class, year, key, mode, name)) %>% 
    #select(!c(pop.class, year, name)) %>% 
    select(!c(pop.class, name))
songs_reg_test  <- testing(reg_data_split)  %>% 
    #select(!c(pop.class, year, key, mode, name)) %>% 
    #select(!c(pop.class, year, name)) %>% 
    select(!c(pop.class, name))

In [ ]:
songs_reg_cv <- rsample::vfold_cv(songs_reg_train, v=10, repeats=1)

In [ ]:
threshold_pred <- function(results) {
    results %>%
        mutate(D=ifelse(.pred<20, TRUE, FALSE)) %>%
        mutate(C=ifelse(.pred<40 & .pred>=20, TRUE, FALSE)) %>%
        mutate(B=ifelse(.pred<60 & .pred>=40, TRUE, FALSE)) %>%
        mutate(A=ifelse(.pred>=60, TRUE, FALSE)) %>%
        pivot_longer(cols=D:A, names_to = '.pred_class', values_to='dummy') %>%
        mutate(.pred_class=factor(.pred_class, levels=c('A', 'B', 'C', 'D'))) %>%
        filter(dummy) %>%
        select(!dummy)
}

In [ ]:
#lin_reg_grid <- tibble(penalty = 10^seq(-2,1, length.out = 30))
lin_reg_grid <- expand.grid(penalty = 10^seq(-1, 1, length.out = 20), mixture=seq(0,1, length = 3))

lin_reg_rec <- recipe(popularity ~ ., data = songs_reg_train) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>%
    step_dummy(all_nominal(), -all_outcomes())

lin_reg_mod <- linear_reg(penalty = tune(), mixture = tune()) %>%
    set_engine("glmnet")

lin_reg_wf <- workflow() %>%
    add_recipe(lin_reg_rec) %>%
    add_model(lin_reg_mod) #%>% add_formula(popularity ~ .) 

In [ ]:
lin_reg_fit <- lin_reg_wf %>%
    tune_grid(
        resamples = songs_reg_cv,
        grid = lin_reg_grid,
        control = control_grid(verbose=TRUE)
    )

In [ ]:
fig(12,8)
lin_reg_fit %>%
    collect_metrics() %>%
    ggplot(aes(penalty, mean, color = factor(mixture))) +
    #geom_errorbar(aes(ymin = mean - std_err,
    #                  ymax = mean + std_err), alpha = 0.5) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    scale_x_log10() +
    labs(x='Lasso penalty', y='Metric', color='Mixture') +
    theme(text=element_text(size=16))

In [ ]:
lin_reg_fit %>%
    select_by_one_std_err(metric="rmse", desc(penalty))
    #select_by_one_std_err(metric="rmse", penalty)

In [ ]:
lin_reg_best <- lin_reg_fit %>%
    select_by_one_std_err(metric="rmse", desc(penalty), desc(mixture))
    #select_by_one_std_err(metric="rmse", desc(penalty))
    #select_best(metric="rmse")

var_importance <- lin_reg_wf %>% 
    finalize_workflow(lin_reg_best) %>%
    fit(songs_reg_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    slice(-1) %>%
    filter(abs(estimate) > 0) %>%
    select(term, estimate) %>%
    mutate(abs_estimate=abs(estimate), sign=estimate>0) %>%
    arrange(desc(abs_estimate))

fig(8,6)
ggplot(var_importance) +
    geom_bar(stat='identity', aes(reorder(term, abs_estimate), abs_estimate, fill=sign)) +
    scale_x_discrete(breaks=var_importance$term) +
    labs(x='', y='Importance') +
    coord_flip() +
    guides(fill='none') 

In [ ]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")
lin_reg_wf %>% 
    finalize_workflow(lin_reg_best) %>%
    fit(songs_reg_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    filter(abs(estimate) > 0) %>%
    select(term, estimate) %>%
    xtable(type = "latex") %>%
print(file = "lm.tex")

In [ ]:
lin_reg_best_fit <- lin_reg_wf %>% 
    finalize_workflow(lin_reg_best) %>%
    fit(data=songs_reg_train)


lin_reg_results <- lin_reg_best_fit %>%
    #predict(new_data = songs_reg_test) %>%
    predict(new_data = songs_reg_test) %>%
    bind_cols(testing(reg_data_split), .) %>%
    threshold_pred()

In [ ]:
max_pop <- 88
fig(7,7)
lin_reg_results %>%
    #mutate(.pred=pmax(pmin(.pred, 100), 0)) %>%
    ggplot(aes(.pred, popularity)) + 
    geom_point(alpha=.75) +
    annotate('line', x = c(0, max_pop), y = c(0, max_pop), color='#2FD565', size=1, linetype='dashed') +
    #scale_x_continuous(limits=c(0,100), name='Predicted popularity') +
    #scale_y_continuous(limits=c(0,100), name='Popularity') +
    labs(x='Prediction', y='Popularity') +
    theme(text=element_text(size=16)) +
    coord_equal()

In [ ]:
lin_reg_results %>%
    mutate(residuals = popularity - .pred) %>%
ggplot(aes(.pred, residuals)) +
    geom_point()

### First try

In [ ]:
lowest_rmse <- lasso_grid %>%
    select_by_one_std_err(metric="rmse", penalty)
    #select_best("rmse")

final_lasso <- finalize_workflow(
  wf %>% add_model(tune_spec),
  lowest_rmse
)

In [ ]:
library(vip)

fig(6,4)
final_lasso %>%
  fit(songs_reg_train) %>%
  pull_workflow_fit() %>%
  vi(lambda = lowest_rmse$penalty) %>%
  mutate(
    Importance = abs(Importance),
    Variable = fct_reorder(Variable, Importance)
  ) %>%
  ggplot(aes(x = Importance, y = Variable, fill = Sign)) +
  geom_col() +
  scale_x_continuous(expand = c(0, 0)) +
  labs(y = NULL)

In [ ]:
lin_reg_fit <- last_fit(
    final_lasso,
    reg_data_split
) %>%
    pull_workflow_fit() %>% str()

In [ ]:
fig(5,5)
lin_reg_fit$.predictions[[1]] %>%
    mutate(.pred=pmax(pmin(.pred, 100), 0)) %>%
    ggplot(aes(.pred, popularity)) + 
    geom_point() +
    annotate('line', x = c(0,100), y = c(0,100), color='#2FD565', size=1, linetype='dashed') +
    scale_x_continuous(limits=c(0,100), name='Predicted popularity') +
    scale_y_continuous(limits=c(0,100)) +
    coord_equal()

## Linear SVM

In [ ]:
lin_svm_reg_grid <- tibble(C = 10^seq(-4,1, length.out = 5))

lin_svm_reg_rec <- recipe(popularity ~ ., data = songs_reg_train) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>%
    step_dummy(all_nominal(), -all_outcomes())

svm_reg_mod <- svm_polynomial(C = tune()) %>%
    set_mode("regression") %>%
    set_engine("kernlab")

lin_svm_reg_wf <- workflow() %>%
    add_recipe(lin_svm_reg_rec) %>%
    add_model(lin_svm_reg_mod)

## Decision tree regression

In [ ]:
tree_reg_grid <- tibble(cost_complexity = seq(1e-4, 1e-2, length.out = 30))

tree_reg_rec <- recipe(popularity ~ ., data = songs_reg_train)

tree_reg_mod <- decision_tree(cost_complexity = tune()) %>%
    set_mode("regression") %>%
    set_engine("rpart")

tree_reg_wf <- workflow() %>%
    add_recipe(tree_reg_rec) %>%
    add_model(tree_reg_mod)

In [ ]:
tree_reg_fit <- tree_reg_wf %>%
    tune_grid(
        resamples = songs_reg_cv,
        grid = tree_reg_grid
    )

In [ ]:
fig(10,8)
tree_reg_fit %>%
    collect_metrics() %>%
    ggplot(aes(cost_complexity, mean, color = .metric)) +
    geom_errorbar(aes(ymin = mean - std_err,
                      ymax = mean + std_err), alpha = 0.5, size=1) +
    geom_line(size = 1.5) +
    facet_wrap(~.metric, scales = "free", nrow = 2) +
    #scale_x_log10() +
    labs(x='Cp penalty', y='Metric') +
    theme(legend.position = "none",
          text=element_text(size=16))

In [ ]:
tree_reg_best <- tree_reg_fit %>%
    select_best(metric="rmse")
    #select_by_one_std_err(metric="rmse", penalty)

In [ ]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")
lin_reg_wf %>% 
    finalize_workflow(lin_reg_best) %>%
    fit(songs_reg_train) %>%
    pull_workflow_fit() %>%
    tidy() %>%
    filter(abs(estimate) > 0) %>%
    select(term, estimate) %>%
    xtable(type = "latex") %>%
print(file = "lm.tex")

In [ ]:
tree_reg_best_fit <- tree_reg_wf %>% 
    finalize_workflow(tree_reg_best) %>%
    fit(data=songs_reg_train)

tree_reg_results <- tree_reg_best_fit %>%
    predict(new_data = songs_reg_test) %>%
    bind_cols(testing(reg_data_split), .) %>%
    threshold_pred()

In [ ]:
pred_count <- tree_reg_results %>% 
    count(.pred) %>%
    bind_cols(tree_reg_results %>%
                  group_by(.pred) %>%
                  summarise(min_pop = min(popularity)) %>% 
                  select(min_pop)) %>%
    arrange(.pred)

x_labs <- paste('(N=', pred_count$n, ')', sep='')

In [ ]:
#tree_reg_results %>%
    mutate(D=ifelse(.pred<20, TRUE, FALSE)) %>%
    mutate(C=ifelse(.pred<40 & .pred>=20, TRUE, FALSE)) %>%
    mutate(B=ifelse(.pred<60 & .pred>=40, TRUE, FALSE)) %>%
    mutate(A=ifelse(.pred>=60, TRUE, FALSE)) %>%
    pivot_longer(cols=D:A, names_to = '.pred_class', values_to='dummy') %>%
    filter(dummy) %>%
    select(!dummy)


In [ ]:
max_pop <- 75
fig(10,6)
tree_reg_results %>%
    #mutate(.pred=pmax(pmin(.pred, 100), 0)) %>%
    ggplot(aes(.pred, popularity)) + 
    geom_boxplot(aes(group=.pred), width=5, position='identity', varwidth=TRUE) +
    annotate('line', x = c(0,max_pop), y = c(0,max_pop), color='#2FD565', size=1, linetype='dashed') +
    #annotate('text', x=pred_count$.pred, y=pred_count$min_pop-2, label=x_labs)
    scale_x_continuous(limits=c(0, max_pop), breaks=25*c(0:3)) +
    labs(x='Prediction', y='Popularity') +
    theme(text=element_text(size=16))

In [ ]:
fig(6,6)
plot_conf_mat(tree_reg_results)

tree_reg_results %>%
    yardstick::accuracy(pop.class, .pred_class)

tree_reg_results %>%
    yardstick::rmse(popularity, .pred)

In [ ]:
library(rpart.plot)
fig(10,6)
rpart.plot(tree_reg_best_fit$fit$fit$fit)

In [ ]:
nn_reg_rec <- recipe(popularity ~ ., data = songs_reg_train) %>%
    step_normalize(all_numeric(), -all_outcomes()) %>%
    step_dummy(all_nominal(), -all_outcomes())

nn_reg_mod <- mlp(hidden_units = 16, activation='relu', penalty=0.01, epochs=200) %>%
    set_mode("regression") %>%
    set_engine("keras", verbose=2)

nn_reg_wf <- workflow() %>%
    add_recipe(nn_reg_rec) %>%
    add_model(nn_reg_mod)

In [ ]:
nn_reg_mod_fit <- nn_reg_wf %>%
    fit(data=songs_reg_train)

In [ ]:
nn_reg_mod_results <- nn_reg_mod_fit %>%
    predict(new_data = songs_reg_test) %>%
    bind_cols(testing(reg_data_split), .) %>%
    threshold_pred()

In [ ]:
nn_reg_mod_results %>%
    yardstick::rmse(popularity, .pred)

nn_reg_mod_results %>%
    yardstick::accuracy(pop.class, .pred_class)
    

plot_conf_mat(nn_reg_mod_results)

In [ ]:
max_pop <- 88
fig(7,7)
nn_reg_mod_results %>%
    ggplot(aes(.pred, popularity)) + 
    geom_point( alpha=.6) +
    annotate('line', x = c(0, max_pop), y = c(0, max_pop), color='#2FD565', size=1, linetype='dashed') +
    #scale_x_continuous(limits=c(0,100), name='Predicted popularity') +
    #scale_y_continuous(limits=c(0,100), name='Popularity') +
    coord_equal()